# Import Libraries

In [80]:
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
from scipy.stats import chi2_contingency
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Import Data

In [2]:
data = pd.read_csv('fashion_data_on_Instagram.csv')
data.set_index('UserId', inplace = True)
data.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Clean Data

### Let's See if there are any NaNs in the data

In [17]:
set(data['Hashtags'].isna())

{False, True}

In [18]:
set(data['Caption'].isna())

{False, True}

In [50]:
set(data['Comments '].isna())

{False}

##### So there exist NaN values in Hashtags and Captions. There are no other columns that contain NaN.

In [55]:
data.isna().groupby('Hashtags')['Followings'].count()

Hashtags
False    24606
True       146
Name: Followings, dtype: int64

##### There are 146 NaN rows in hashtags. It is possible that this is simply due to the fact that some posts do not have hashtags.

In [57]:
data.isna().groupby('Caption')['Followings'].count()

Caption
False    24717
True        35
Name: Followings, dtype: int64

##### There are 35 NaN rows in Caption

### Maybe the caption or hashtag information is missing because there is either only a caption or only hashtags....

In [69]:
hashtag_nan = list(data['Hashtags'].isna())
caption_nan = list(data['Caption'].isna())

In [73]:
count = 0
for i in range(len(hashtag_nan)):
    if hashtag_nan[i] and caption_nan[i]:
        count += 1
print('The number of times both hashtag_nan and caption_nan is true is', count)

The number of times both hashtag_nan and caption_nan is true is 18


##### So we have missing hashtag both 18/35 times that we have a missing caption. So it is not exactly correlated. Maybe we can just drop the rows that have missing caption or hashtag if we need to use these, or claim that there was not hashtag or caption for these rows.

### I want to try and see if I can make brand category numeric

In [115]:
brand_category = pd.get_dummies(data['BrandCategory'])

In [116]:
brand_category.head()

,Designer,High street,Mega couture,Small couture
UserId,,,,
1171579752855683619_212070047,0,1,0,0
1171594777274371222_176762322,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171407552643586413_581125501,0,1,0,0
1171508187966229230_2797323089,0,1,0,0


In [117]:
data_new = pd.concat([data, brand_category], sort = False, axis = 1)
data_new.head()

,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,Comments,CreationTime,Link,Selfie,BodySnap,Marketing,ProductOnly,NonFashion,Face,Logo,BrandLogo,Smile,Outdoor,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise,Designer,High street,Mega couture,Small couture
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,0,1453883211,https://www.instagram.com/p/BBCSZPzAMIj/,0.000885,0.336438,0.741220,0.009454,0.001827,0.839466,0.890231,0.292843,0.062638,0.012164,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181,0,1,0,0
1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,0,1453885002,https://www.instagram.com/p/BBCVz4YJEyW/,0.003912,0.995503,0.001728,0.002125,0.002438,0.480598,0.550629,0.063139,0.126848,0.004330,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,3,1453862683,https://www.instagram.com/p/BBBrPZ2h9lt/,0.015774,0.963880,0.025719,0.027023,0.003409,0.283399,0.095724,0.037635,0.107896,0.712017,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0
1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,1,1453874680,https://www.instagram.com/p/BBCCH1zMEru/,0.002021,0.014188,0.028351,0.418125,0.601519,0.010840,0.994191,0.332847,0.007911,0.002386,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0


##### Now the brand category is represented in the last four columns of the dataframe with one_hot_encoding.

### In order to model with these we need to normalize the variables

In [119]:
for col in data_new.columns:
    if col not in ['BrandName', 'BrandCategory','Hashtags', 'Caption', 'ImgURL', 'Likes', 'Comments ', 'CreationTime','Link','Designer',
       'High street', 'Mega couture', 'Small couture']:
        data_new[col] = data_new[col]/np.linalg.norm(np.array(data_new[col], dtype = float))

# Model with Numeric Data Only

First, lets try to build a model with only the numeric data. This will be easy to do and will allow us to understand whether any of these numeric factors influence the number of likes/comments a user gets. 

### Likes Model

In [121]:
X = data_new[['Followings', 'Followers', 'MediaCount', 'Selfie', 'BodySnap', 'Marketing', 'ProductOnly', 'NonFashion',
       'Face', 'Logo', 'BrandLogo', 'Smile', 'Outdoor', 'NumberOfPeople',
       'NumberOfFashionProduct', 'Anger', 'Contempt', 'Disgust', 'Fear',
       'Happiness', 'Neutral', 'Sadness', 'Surprise', 'Designer',
       'High street', 'Mega couture', 'Small couture']]
y = data_new['Likes']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Let's use a Lasso Regression model to regress based on the likes that we receive. I want to use lasso because lasso will send the unimportant variables to zero allowing us to see which features are important. 

In [152]:
lasso = Lasso(alpha = 0.0001, max_iter = 10000).fit(X_train, y_train)

In [153]:
predictions = lasso.predict((X_test))
r2_score(y_test, predictions)

0.2984467039649339

Maybe ridge regression will work better?

In [180]:
ridge = Ridge(alpha = 0.0001).fit(X_train, y_train)

In [181]:
predictions = ridge.predict((X_test))
r2_score(y_test, predictions)

0.29844937687206075

So ridge regression is really no different...

### Try a Deep Learning Model

In [196]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
def our_model():
    model = Sequential()

    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=64, activation='relu', input_dim = 27))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [199]:
estimator = KerasRegressor(build_fn=our_model, epochs=100, batch_size=5, verbose=1)

In [200]:
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
22276/22276 [==============================] - 3s 150us/step - loss: 749465.9903
Epoch 2/100
22276/22276 [==============================] - 3s 145us/step - loss: 747334.5938
Epoch 3/100
22276/22276 [==============================] - 3s 144us/step - loss: 743619.6023
Epoch 4/100
22276/22276 [==============================] - 3s 145us/step - loss: 735755.6552
Epoch 5/100
22276/22276 [==============================] - 3s 145us/step - loss: 724785.2448
Epoch 6/100
22276/22276 [==============================] - 3s 145us/step - loss: 713068.3527
Epoch 7/100
22276/22276 [==============================] - 3s 147us/step - loss: 701388.7006
Epoch 8/100
22276/22276 [==============================] - 3s 145us/step - loss: 675024.6562
Epoch 9/100
22276/22276 [==============================] - 3s 145us/step - loss: 646445.6853
Epoch 10/100
22276/22276 [==============================] - 3s 146us/step - loss: 616160.3869
Epoch 11/100
22276/22276 [==============================] - 3s 146us/

22276/22276 [==============================] - 3s 143us/step - loss: 173643.0788
Epoch 89/100
22276/22276 [==============================] - 3s 142us/step - loss: 176377.2238
Epoch 90/100
22276/22276 [==============================] - 3s 143us/step - loss: 174699.3867
Epoch 91/100
22276/22276 [==============================] - 3s 142us/step - loss: 170253.2199
Epoch 92/100
22276/22276 [==============================] - 3s 145us/step - loss: 173962.9639
Epoch 93/100
22276/22276 [==============================] - 3s 147us/step - loss: 173113.9813
Epoch 94/100
22276/22276 [==============================] - 3s 144us/step - loss: 170620.3553
Epoch 95/100
22276/22276 [==============================] - 3s 147us/step - loss: 170576.0772
Epoch 96/100
22276/22276 [==============================] - 3s 146us/step - loss: 172158.8285
Epoch 97/100
22276/22276 [==============================] - 3s 148us/step - loss: 168281.7652
Epoch 98/100
22276/22276 [==============================] - 3s 144us/step

22276/22276 [==============================] - 3s 153us/step - loss: 153584.6739
Epoch 75/100
22276/22276 [==============================] - 3s 155us/step - loss: 153447.1933
Epoch 76/100
22276/22276 [==============================] - 3s 156us/step - loss: 151873.9856
Epoch 77/100
22276/22276 [==============================] - 3s 152us/step - loss: 152515.0099
Epoch 78/100
22276/22276 [==============================] - 3s 152us/step - loss: 149788.6592
Epoch 79/100
22276/22276 [==============================] - 3s 153us/step - loss: 152353.0325
Epoch 80/100
22276/22276 [==============================] - 3s 153us/step - loss: 148072.8367
Epoch 81/100
22276/22276 [==============================] - 3s 154us/step - loss: 147835.8453
Epoch 82/100
22276/22276 [==============================] - 3s 152us/step - loss: 148991.0026
Epoch 83/100
22276/22276 [==============================] - 3s 153us/step - loss: 148399.3776
Epoch 84/100
22276/22276 [==============================] - 3s 152us/step

22277/22277 [==============================] - 3s 142us/step - loss: 185391.8155
Epoch 61/100
22277/22277 [==============================] - 3s 142us/step - loss: 177477.0507
Epoch 62/100
22277/22277 [==============================] - 3s 142us/step - loss: 185369.5194
Epoch 63/100
22277/22277 [==============================] - 3s 141us/step - loss: 180074.4373
Epoch 64/100
22277/22277 [==============================] - 3s 142us/step - loss: 179035.0224
Epoch 65/100
22277/22277 [==============================] - 3s 141us/step - loss: 178474.5209
Epoch 66/100
22277/22277 [==============================] - 3s 141us/step - loss: 179606.0329
Epoch 67/100
22277/22277 [==============================] - 3s 142us/step - loss: 173787.6923
Epoch 68/100
22277/22277 [==============================] - 3s 141us/step - loss: 173765.7663
Epoch 69/100
22277/22277 [==============================] - 3s 142us/step - loss: 175329.1791
Epoch 70/100
22277/22277 [==============================] - 3s 143us/step

22277/22277 [==============================] - 3s 142us/step - loss: 204079.1722
Epoch 47/100
22277/22277 [==============================] - 3s 142us/step - loss: 197280.7900
Epoch 48/100
22277/22277 [==============================] - 3s 141us/step - loss: 199700.3810
Epoch 49/100
22277/22277 [==============================] - 3s 145us/step - loss: 206585.5306
Epoch 50/100
22277/22277 [==============================] - 3s 141us/step - loss: 200730.6343
Epoch 51/100
22277/22277 [==============================] - 3s 140us/step - loss: 194517.0521
Epoch 52/100
22277/22277 [==============================] - 3s 141us/step - loss: 192170.8370
Epoch 53/100
22277/22277 [==============================] - 3s 141us/step - loss: 200094.8805
Epoch 54/100
22277/22277 [==============================] - 3s 141us/step - loss: 194900.0054
Epoch 55/100
22277/22277 [==============================] - 3s 141us/step - loss: 199938.7132
Epoch 56/100
22277/22277 [==============================] - 3s 141us/step

...

22277/22277 [==============================] - 3s 144us/step - loss: 801312.1068
Epoch 5/100
22277/22277 [==============================] - 3s 144us/step - loss: 787738.1055
Epoch 6/100
22277/22277 [==============================] - 3s 144us/step - loss: 755791.8363
Epoch 7/100
22277/22277 [==============================] - 3s 145us/step - loss: 709096.3107
Epoch 8/100
22277/22277 [==============================] - 3s 144us/step - loss: 667857.6748
Epoch 9/100
22277/22277 [==============================] - 3s 150us/step - loss: 614010.5075
Epoch 10/100
22277/22277 [==============================] - 3s 150us/step - loss: 530011.4843
Epoch 11/100
22277/22277 [==============================] - 3s 149us/step - loss: 452785.5279
Epoch 12/100
22277/22277 [==============================] - 3s 150us/step - loss: 378144.4613
Epoch 13/100
22277/22277 [==============================] - 3s 149us/step - loss: 333815.1155
Epoch 14/100
22277/22277 [==============================] - 3s 151us/step - lo

22277/22277 [==============================] - 3s 148us/step - loss: 171995.7011
Epoch 92/100
22277/22277 [==============================] - 3s 148us/step - loss: 171655.3025
Epoch 93/100
22277/22277 [==============================] - 3s 148us/step - loss: 179616.8631
Epoch 94/100
22277/22277 [==============================] - 3s 149us/step - loss: 178925.9320
Epoch 95/100
22277/22277 [==============================] - 3s 148us/step - loss: 197385.9772
Epoch 96/100
22277/22277 [==============================] - 3s 148us/step - loss: 176877.2977
Epoch 97/100
22277/22277 [==============================] - 3s 149us/step - loss: 172699.4262
Epoch 98/100
22277/22277 [==============================] - 3s 148us/step - loss: 171369.4732
Epoch 99/100
22277/22277 [==============================] - 3s 149us/step - loss: 173854.5802
Epoch 100/100
2475/2475 [==============================] - 0s 48us/step
Epoch 1/100
22277/22277 [==============================] - 3s 154us/step - loss: 347792.6568
Ep

22277/22277 [==============================] - 3s 150us/step - loss: 157003.5513
Epoch 78/100
22277/22277 [==============================] - 3s 150us/step - loss: 156605.9715
Epoch 79/100
22277/22277 [==============================] - 4s 160us/step - loss: 157038.0433
Epoch 80/100
22277/22277 [==============================] - 3s 153us/step - loss: 160150.5902
Epoch 81/100
22277/22277 [==============================] - 3s 150us/step - loss: 157470.7213
Epoch 82/100
22277/22277 [==============================] - 3s 149us/step - loss: 157477.2787
Epoch 83/100
22277/22277 [==============================] - 3s 150us/step - loss: 156435.3353
Epoch 84/100
22277/22277 [==============================] - 3s 149us/step - loss: 157839.3326
Epoch 85/100
22277/22277 [==============================] - 3s 149us/step - loss: 157354.6017
Epoch 86/100
22277/22277 [==============================] - 3s 150us/step - loss: 154536.1420
Epoch 87/100
22277/22277 [==============================] - 3s 149us/step

22277/22277 [==============================] - 3s 142us/step - loss: 183425.4462
Epoch 64/100
22277/22277 [==============================] - 3s 141us/step - loss: 175396.6255
Epoch 65/100
22277/22277 [==============================] - 3s 142us/step - loss: 177404.7795
Epoch 66/100
22277/22277 [==============================] - 3s 141us/step - loss: 184033.6161
Epoch 67/100
22277/22277 [==============================] - 3s 142us/step - loss: 177129.5851
Epoch 68/100
22277/22277 [==============================] - 3s 142us/step - loss: 182581.4080
Epoch 69/100
22277/22277 [==============================] - 3s 141us/step - loss: 178265.9927
Epoch 70/100
22277/22277 [==============================] - 3s 143us/step - loss: 182935.1044
Epoch 71/100
22277/22277 [==============================] - 3s 142us/step - loss: 177244.9479
Epoch 72/100
22277/22277 [==============================] - 3s 142us/step - loss: 175097.0414
Epoch 73/100
22277/22277 [==============================] - 3s 142us/step

22277/22277 [==============================] - 3s 148us/step - loss: 170945.3007
Epoch 50/100
22277/22277 [==============================] - 3s 148us/step - loss: 170687.9132
Epoch 51/100
22277/22277 [==============================] - 3s 148us/step - loss: 167817.2045
Epoch 52/100
22277/22277 [==============================] - 3s 147us/step - loss: 171866.7413
Epoch 53/100
22277/22277 [==============================] - 3s 147us/step - loss: 165661.9348
Epoch 54/100
22277/22277 [==============================] - 3s 147us/step - loss: 168892.7354
Epoch 55/100
22277/22277 [==============================] - 3s 148us/step - loss: 165737.0784
Epoch 56/100
22277/22277 [==============================] - 3s 148us/step - loss: 166719.7183
Epoch 57/100
22277/22277 [==============================] - 3s 147us/step - loss: 162208.4169
Epoch 58/100
22277/22277 [==============================] - 3s 148us/step - loss: 169050.3005
Epoch 59/100
22277/22277 [==============================] - 3s 147us/step

Maybe for the neural network model to work we need to normalize the outputs as well??? I have only normalized the input space. Maybe we need to investigate how the input space was normalized. Maybe the categorical variables that are represented by 1's and 0's at the end of the dataframe need to be normalized?